# 自动站数据转换

In [14]:
import pandas as pd
import numpy as np
import xarray as xr
station_Mws = pd.read_csv("Mws2022.csv", keep_default_na=-9999)
station_Aws = pd.read_csv("Aws2022.csv", keep_default_na=-9999)
station_all = pd.concat([station_Aws,station_Mws])

grouped = station_all.groupby('IIiii')

# 根据time lat lon RR 对数据进行转换



# Class版的数据处理

In [34]:
import pandas as pd
import numpy as np
from math import isnan
sql = 'test'  

class sql_data:
    def __init__(self, sql):
        self.sql = sql# 传进来的参数
        station_Mws = pd.read_csv("Mws2022.csv")
        station_Aws = pd.read_csv("Aws2022.csv")
        self.station_all = pd.concat([station_Aws,station_Mws])
        # 数据
        self.grouped_county = self.station_all.groupby('county')
        self.grouped_IIiii = self.station_all.groupby('IIiii')
        self.timecounts = len(self.grouped_IIiii.get_group(58660)['tTime'])
        # 测试数据
        print(self.timecounts)
    def comput_county(self):
        '计算面最大雨强、累计降水、最高、最低气温'
        self.station_county_comput = []
        for i in self.grouped_county.size().index:  
            data= self.grouped_county.get_group(i)
            data['VV'].replace(-9999,np.nan,inplace=True)
            data['RR'].replace(-9999,np.nan,inplace=True)
            data['Tn'].replace(-9999,np.nan,inplace=True)
            data['Tx'].replace(-9999,np.nan,inplace=True)
            dic = {}
            dic['county'] = str(i)
            dic['RR'] = data['RR'].mean()*self.timecounts/10.0
            dic['RMax'] = data['RR'].max()/10.0
            dic['Tx'] = data['Tx'].max()/10.0
            dic['Tn'] = data['Tn'].min()/10.0 
#             print(dic)
            self.station_county_comput.append(dic)
        tmp_max_County = []
        tmp_min_County = []
        RR_County = []
        for i in self.station_county_comput:
            tmp_max_County.append({"name":i['county'],"value":i['Tx']})
            tmp_min_County.append({"name":i['county'],"value":i['Tn']})
            RR_County.append({"name":i['county'],"value":i['RR']})
        return RR_County,tmp_max_County,tmp_min_County

    def comput_IIiii(self):
        self.station_dot_comput = {} 
        lat = []
        lon = []
        tx = []
        tn = []
        rr = [] 
        rx = []
        county = []
        name = []
        vv = []
        town = []
        vv_min = " "#{"name":[],"value":0.0}
        vv_value = 1000.0
        station_RR_small = 0.0
        station_RR_mid = 0.0
        station_RR_big = 0.0
        station_RR_huge = 0.0
        station_RR_bighuge = 0.0
        station_RR_more = 0.0
        station_VV_small = 0.0
        station_VV_mid = 0.0
        station_VV_big = 0.0
        station_VV_huge = 0.0
        station_VV_more = 0.0
        station_vv = []
        VV_scatter_list = []
        # 指标站数据
        nation_station = ['58660','58666','K8505','K8206','58665','58559','58655','K8271','58662','58653']
        temp_scatter_list = ['58559','K8705','K8706','58652','K8903','58568','K8818','58662','K8821','58660',
                             '58653','K8609','K8505','58667','58664','K8413','58655','K8282','K8217','K8201','K8301','58665']
        for i in self.grouped_IIiii.size().index:
            data= self.grouped_IIiii.get_group(i)
            #print(data)
            data['VV'].replace(-9999,np.nan,inplace=True)
            data['RR'].replace(-9999,np.nan,inplace=True)
            data['Tn'].replace(-9999,np.nan,inplace=True)
            data['Tx'].replace(-9999,np.nan,inplace=True)
            dic = {}
            dic['IIiii'] = data['IIiii'].iloc[0]
            dic['tTime'] = data['tTime'].tolist()
            dic['StationName'] = data['StationName'].iloc[0]
            dic['county'] = data['county'].iloc[0]
            dic['lat'] = data['lat'].iloc[0]
            dic['lon'] = data['lon'].iloc[0]
            dic['Town'] = data['Town'].iloc[0]
            dic['fFy'] = data['fFy'].max()           
            dic['dFy'] = data[data['fFy']== data['fFy'].max()]['dFy'].iloc[0]
            dic['T'] = data['T'].tolist()
            dic['VList'] = data['VV'].tolist()
            dic['Tx'] = data['Tx'].max()/10.0
                        
            dic['Tn'] = data['Tn'].min()/10.0
            if not isnan(data['VV'].min()):
                #统计能见度自动站名称
                station_vv.append(data['IIiii'].iloc[0])
                if data['VV'].min() < vv_value:
#                     print(data['IIiii'].iloc[0])
                    vv_min=  str(data['IIiii'].iloc[0])
                    vv_value = data['VV'].min()
            dic['VV'] = data['VV'].min()
            dic['RR'] = data['RR'].sum()
            dic['RMax'] = data['RR'].max()
            # 降水分级别
            if not isnan(data['RR'].sum()):
                value_rsum = data['RR'].sum()
                if value_rsum >=0 and value_rsum <10:
                    station_RR_small =  station_RR_small +1   
                elif value_rsum >=10 and value_rsum <25:
                    station_RR_mid =  station_RR_mid+1   
                elif value_rsum >=25 and value_rsum <50:
                    station_RR_big =  station_RR_big +1   
                elif value_rsum >=50 and value_rsum <100:
                    station_RR_huge =  station_RR_huge+1   
                elif value_rsum >=100 and value_rsum <250:
                    station_RR_more =  station_RR_bighuge+1 
                else:
                    station_RR_more =  station_RR_more+1
            # 能见度分级别  
            if not isnan(data['VV'].min()):
                VV_scatter_list.append(data['IIiii'].iloc[0])
                value_VV = data['VV'].min()
#                 print(value_VV)
                if value_VV >=0 and value_VV <50:
                    station_VV_small =  station_VV_small +1   
                elif value_VV >=50 and value_VV <200:
                    station_VV_mid =  station_VV_mid+1   
                elif value_VV >=200 and value_VV <500:
                    station_VV_big =  station_VV_big +1   
                elif value_VV >=500 and value_VV <1000:
                    station_VV_huge =  station_VV_huge+1   
                else: 
                    station_VV_more =  station_VV_more+1 

            lat.append(data['lat'].iloc[0])
            town.append(data['Town'].iloc[0])
            lon.append(data['lon'].iloc[0])
            county.append(data['county'].iloc[0])
            name.append(data['IIiii'].iloc[0])
            tx.append(data['Tx'].max()/10.0)
            tn.append(data['Tn'].max()/10.0)
            rr.append(data['RR'].sum()/10.0)
            rx.append(data['RR'].max()/10.0)
#             print(dic)
            self.station_dot_comput[str(i)] = dic 
#         print(lat)
        #排序数据
        data_rx =  pd.DataFrame()
        data_rx['name'] = name 
        data_rx['county'] = county
        data_rx['town'] = town
        data_rx['rx'] = rx 
#         print(data_rx.sort_values(by =['rx'],ascending = [False]))
        # 按照累计降水进行排序
        data_rsum =  pd.DataFrame()
        data_rsum['name'] = name 
        data_rsum['county'] = county
        data_rsum['town'] = town
        data_rsum['rsum'] = rr 
        data_rsum['index']=data_rsum['rsum'].rank(ascending=0,method='dense')
        data_rr_sum = data_rsum.sort_values(by =['rsum'],ascending = [False])
#         data_rsum['index'] = [a for i in ]
#         print(data_rx.sort_values(by =['rx'],ascending = [False]))
        # 最大值对应的站点序列
        data_vv = vv_min
        #print("最低能见度",data_vv,vv_min)
        data_vvmin =  pd.DataFrame()
        data_vvmin['tTime']= self.station_dot_comput[data_vv]['tTime']
        data_vvmin['VV']= self.station_dot_comput[data_vv]['VList']
        #print("data:",data_vvmin)
        # 降水分级
        RR_station_rank = [
            { "value": station_RR_small, "name": '小雨' },
            { "value": station_RR_mid, "name": '中雨' },
            { "value": station_RR_big, "name": '大雨' },
            { "value": station_RR_huge, "name": '暴雨' },
            { "value": station_RR_bighuge, "name": '大暴雨' },
            { "value": station_RR_more, "name": '特大暴雨' }
        ]
        tmp_station_bar = []
        tmp_station_bar.append(['product', '最高气温','最低气温'])
        RR_station_bar = []
        RR_station_bar.append(['product', '累计降水','最大雨强'])
        # 计算指标站nation_station的要素值
        for i in nation_station:
            tmp_station_bar.append([self.station_dot_comput[i]['StationName'], self.station_dot_comput[i]['Tx'],self.station_dot_comput[i]['Tn']])
            RR_station_bar.append([self.station_dot_comput[i]['StationName'], self.station_dot_comput[i]['RR'],self.station_dot_comput[i]['RMax']])
  
        # 返回站点气温数据
        tmp_min_scatter = []
        tmp_max_scatter = []
        for i in temp_scatter_list:
            dic_temp_max = {"value":[],"url":"",}
            dic_temp_max['value'].append(self.station_dot_comput[i]['lon'])
            dic_temp_max['value'].append(self.station_dot_comput[i]['lat'])
            dic_temp_max['value'].append(self.station_dot_comput[i]['Tx'])
            dic_temp_max['url'] ="station/"+str(self.station_dot_comput[i]['IIiii'])
            dic_temp_max['name'] = str(self.station_dot_comput[i]['StationName'])
            dic_temp_min = {"value":[],"url":""}
            dic_temp_min['value'].append(self.station_dot_comput[i]['lon'])
            dic_temp_min['value'].append(self.station_dot_comput[i]['lat'])
            dic_temp_min['value'].append(self.station_dot_comput[i]['Tn'])
            dic_temp_min['url'] ="station/"+str(self.station_dot_comput[i]['IIiii'])
            dic_temp_min['name'] = str(self.station_dot_comput[i]['StationName'])
            tmp_max_scatter.append(dic_temp_max)
            tmp_min_scatter.append(dic_temp_min)
        print("能见度",VV_scatter_list)
        # 返回站点能见度数据
        VV_min_scatter = []
        for i in VV_scatter_list:
            dic_VV_min = {"value":[],"url":""}
            dic_VV_min['value'].append(self.station_dot_comput[str(i)]['lon'])
            dic_VV_min['value'].append(self.station_dot_comput[str(i)]['lat'])
            dic_VV_min['value'].append(self.station_dot_comput[str(i)]['VV'])
            dic_VV_min['url'] ="station/"+str(self.station_dot_comput[str(i)]['IIiii'])
            dic_VV_min['name'] = str(self.station_dot_comput[str(i)]['StationName'])
            VV_min_scatter.append(dic_VV_min) 
        # 能见度分级
        VV_station_rank = [
            { "value": station_VV_small, "name": '强浓雾' },
            { "value": station_VV_mid, "name": '浓雾' },
            { "value": station_VV_big, "name": '大雾' },
            { "value": station_VV_huge, "name": '雾' },
            { "value": station_VV_more, "name": '轻雾' }
        ]
        return data_rr_sum,RR_station_rank,RR_station_bar,tmp_station_bar,tmp_min_scatter,tmp_max_scatter,data_vvmin,VV_min_scatter,VV_station_rank
        # 计算scatter
#         return lat,lon,tx,tn,rr,rx,vv,data_rx.sort_values(by =['rx'],ascending = [False]),data_rsum,data_vvmin
    

    

RR_County,tmp_max_County,tmp_min_County =  sql_data(sql).comput_county()   
RR_sum,RR_station_rank ,RR_station_bar,tmp_station_bar,tmp_min_scatter,tmp_max_scatter,data_vvmin,VV_min_scatter,VV_station_rank = sql_data(sql).comput_IIiii()

# a =  sql_data(sql).station_all
%time

733
733
能见度 [58559, 58568, 58652, 58655, 58660, 58661, 58662, 58663, 58664, 58665, 58666, 58667, 58668, 58669, 'K8106', 'K8111', 'K8114', 'K8116', 'K8117', 'K8125', 'K8126', 'K8201', 'K8205', 'K8210', 'K8217', 'K8226', 'K8303', 'K8315', 'K8316', 'K8318', 'K8320', 'K8322', 'K8407', 'K8409', 'K8413', 'K8425', 'K8427', 'K8447', 'K8466', 'K8502', 'K8504', 'K8510', 'K8513', 'K8522', 'K8528', 'K8530', 'K8611', 'K8627', 'K8629', 'K8650', 'K8655', 'K8674', 'K8675', 'K8718', 'K8719', 'K8802', 'K8803', 'K8807', 'K8810', 'K8813', 'K8820', 'K8821', 'K8829', 'K8841', 'K8843', 'K8860', 'K8862', 'K8872', 'K8873', 'K8929', 'K8951']
CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 3.81 µs


# 降水的分类

24小时内降雨量﹤0.1毫米之间为微量降雨（零星小雨）；

12小时内降雨量0.1～4.9毫米或24小时内降雨量0.1～9.9毫米之间为小雨；

12小时内降雨量5.0～14.9毫米或24小时内降雨量在10～24.9毫米之间为中雨；

12小时内降雨量15.0.1～29.9毫米或24小时内降雨量在25.0～49.9毫米之间为大雨；

12小时内降雨量30.0～69.9毫米或24小时内降雨量在50.0～99.9毫米之间为暴雨；

12小时内降雨量70.0～139.9毫米或24小时内降雨量在100.0～249.9毫米之间为大暴雨；

12小时内降雨量≥140.0毫米或24小时内降雨量≥250.0毫米之间为特大暴雨。

# 浓雾的分类

大雾红色预警发布的标准是，预计未来24小时内有3个及以上省（自治区、直辖市）的部分地区可能出现能见度不足200米的强浓雾，且有成片的（覆盖5个及以上相邻的国家基本气象站或国家基准气候站）能见度小于50米的雾；或者已经出现并可能持续。

大雾橙色预警发布的标准是，预计24小时内3个及以上省（自治区、直辖市）可能出现能见度小于500米的雾，且有成片的（覆盖5个及以上相邻的国家基本气象站或国家基准气候站）能见度小于50米的雾；或者已经出现并可能持续。

大雾黄色预警发布的标准是，预计24小时内3个及以上省（自治区、直辖市）可能出现能见度小于1000米的雾，且有成片的（覆盖5个及以上相邻的国家基本气象站或国家基准气候站）能见度小于200米的雾；或者已经出现并可能持续。

In [12]:
RR_station_rank

[{'value': 37.0, 'name': '小雨'},
 {'value': 0.0, 'name': '中雨'},
 {'value': 0.0, 'name': '大雨'},
 {'value': 0.0, 'name': '暴雨'},
 {'value': 0.0, 'name': '大暴雨'},
 {'value': 450.0, 'name': '特大暴雨'}]

In [16]:
RR_station_bar

[['product', '累计降水', '最大雨强'],
 ['临海', 881, 56],
 ['大陈', 1017, 76],
 ['玉城', 1136, 85],
 ['富山', 1562, 66],
 ['洪家', 876, 67],
 ['天台', 1087, 43],
 ['黄岩站', 259, 29],
 ['外金', 1290, 63],
 ['牛山', 880, 45],
 ['括苍山', 754, 37]]

In [17]:
tmp_station_bar

[['product', '最高气温', '最低气温'],
 ['临海', 17.6, -2.4],
 ['大陈', 12.3, 3.7],
 ['玉城', 16.7, 2.0],
 ['富山', 14.8, -4.0],
 ['洪家', 16.1, 1.2],
 ['天台', 17.1, -3.3],
 ['黄岩站', 10.1, -1.4],
 ['外金', 15.6, -4.4],
 ['牛山', 14.0, 1.4],
 ['括苍山', 9.4, -8.6]]

In [35]:
VV_station_rank

[{'value': 0.0, 'name': '强浓雾'},
 {'value': 18.0, 'name': '浓雾'},
 {'value': 15.0, 'name': '大雾'},
 {'value': 15.0, 'name': '雾'},
 {'value': 23.0, 'name': '轻雾'}]

# 逐行遍历数据

In [ ]:
for row in RR_sum.itertuples():
    index = int(getattr(row, 'index'))
    rsum = getattr(row, 'rsum')
    print(index, rsum) # 输出每一行

In [ ]:
      name county  town   rsum
438  K8914    仙居县   上张乡  195.5
361  K8785    天台县   南屏乡  166.4
71   K8230    黄岩区   平田乡  156.8
50   K8206    黄岩区   富山乡  156.2
377  K8812    三门县  海润街道  155.5
..     ...    ...   ...    ...
315  K8718    天台县   白鹤镇    0.0
328  K8734    天台县   街头镇    0.0
329  K8735    天台县   龙溪乡    0.0
330  K8736    天台县   白鹤镇    0.0
247  K8635    临海市  白水洋镇    0.0